In [ ]:
#   This is filename:  Uhlig_solver_QZ.ipynb

#   It is an abreviated translation of Uhlig's 1997 solve_qz.m
#   code into Python. 

#  The inputs into this notebook are the matrices
#   AA, BB, CC, DD, FF, GG, HH, JJ, KK, LL, MM, NN

#   The output of this notebook are the matices 
#   PP,  QQ,  RR,  SS  and  WW.   
#   All the internediate matrices are available too and 
#   can easily be eyeballed by the user by deleting 
#   the " # "  tags which precede print related commands.


#   The solution method is Harald Uhlig's, as he 
#   described in  " A Tooklit for Analyzing 
#   Nonlinear Dynamic Stochastic Models Easily "  
#   Discussion Paper 101, Federal Reserve Bank 
#   of Minneapolis  June 1995  and his MATLAB 
#   code specifically solve.m and solve_qz.m

#   The section on the QZ decomposition is due to   
#   Chris Sims, as he described in "Solving Linear 
#   Rational Expectations Models,"   Computational 
#   Economics, October 2002,  Vol 20, Issue 1-2, pp 1-20.
#   and  Paul Klein,  as he described in "Using the 
#   Generalized Schur Form to Solve a Multivariate 
#   Linear Rational Expectations Model,"  Journal of Economic 
#   Dynamics and Control, 24 (2000) 405-1423.  

#  Cris Sims' qzdiv.m and qzswitch.m functions in MATLAB
#  were adapted by Kerk Phillips and Yulong Li into Python.

#  I use Uhlig's convention in naming matrices, specifically 
#
#  0 = AA x_(t) +  BB x_(t-1)  +  CC y_(t)  +  DD z_(t)
#
#  0 = E_(t) { FF x_(t+1) +  GG x_(t)  +  HH x_(t-1) +  JJ y_(t+1) +  KK y_(t) +  LL z_(t+1) +  MM z_(t) }
#
#  E_(t) { z_(t+1) } = NN z_(t)  +  E_(t) { e_(t+1) }
#
#  The recursive equililbrium laws of motion are specified as
#
#   x_(t) = PP x_(t-1) + QQ z_(t)
#
#   y_(t) = RR x_(t-1) + SS z_(t)
#
#  z_(t) = NN z_(t-1)  +  e_(t)
#
#  And for convenience
#
#  [ x(t)', y(t)', z(t)' ] = WW [ x(t)', z(t)' ] 
#



import numpy as np
import numpy.matlib as npm
from numpy import hstack, vstack, zeros, dot, eye, kron

import scipy as sp
from scipy import linalg as la
from scipy.linalg import null_space
import scipy.optimize as opt

import matplotlib.pyplot as plt
import pylab

import pandas as pd

import statsmodels.api as sm

from IPython.display import display, HTML

np.set_printoptions(precision=4, suppress=False)
pd.set_option('display.float_format', '{:.4e}'.format)



##  USER INPUT

TOL = 0.0001



AA = np.matrix(AA)
BB = np.matrix(BB)
CC = np.matrix(CC)
DD = np.matrix(DD)
FF = np.matrix(FF)
GG = np.matrix(GG)
HH = np.matrix(HH)
JJ = np.matrix(JJ)
KK = np.matrix(KK)
LL = np.matrix(LL)
MM = np.matrix(MM)
NN = np.matrix(NN)
Sigma =  np.matrix(Sigma_EPS)


m_states = FF.shape[1]
l_equ = CC.shape[0]
n_endog = CC.shape[1]
k_exog = NN.shape[1]

nx = m_states

Varnames =  np.matrix(VARNAMES).reshape(m_states+n_endog+k_exog,1)

SteadyState_mat = np.matrix(SteadyState).reshape(m_states+n_endog+k_exog,1)


##  uhlig_solver_abreviated.ipynb and uhlig_solver_QZ.ipynb 
##  begin the same way

#print('CC shape =', CC.shape)
#print('CC dim =', np.ndim(CC))
#print('CC rank =', np.linalg.matrix_rank(CC))
#print('Rank of CC needs to be = > n_endog')
#print('n_endog =', n_endog)
#print(' ')

CC_plus =  np.linalg.pinv(CC)
CC_plus = np.matrix(CC_plus)
#print('CC_plus= ', CC_plus)
#print('CC_plus shape =', CC_plus.shape)
#print('CC_plus dim =', np.ndim(CC_plus))
#print('CC_plus rank =', np.linalg.matrix_rank(CC_plus))
#print(' ')

CC_T = np.transpose(CC)
#print('CC_T shape =', CC_T.shape)
#print('CC_T dim =', np.ndim(CC_T))
#print('CC_T rank =', np.linalg.matrix_rank(CC_T))
#print(' ')

CC_null = sp.linalg.null_space(CC_T)
#print('CC_null shape =', CC_null.shape)
#print('CC_null dim =', np.ndim(CC_null))
#print('CC_null rank =', np.linalg.matrix_rank(CC_null))
#print(' ')

CC_0 =  np.transpose(CC_null)
#print('CC_0 shape =', CC_0.shape)
#print('CC_0 dim =', np.ndim(CC_0))
#print('CC_0 rank =', np.linalg.matrix_rank(CC_0))
#print(' ')

Gamma_mat = np.vstack((dot(CC_0,AA),
                                         dot(dot(JJ,CC_plus),BB) - GG + dot(dot(KK,CC_plus),AA)))
#print('Gamma_mat = ', Gamma_mat)
#print('Gamma_mat shape =', Gamma_mat.shape)
#print('Gamma_mat dim =', np.ndim(Gamma_mat))
#print('Gamma_mat rank =', np.linalg.matrix_rank(Gamma_mat))
#print(' ')

Psi_mat = np.vstack((zeros((l_equ-n_endog)*m_states).reshape(l_equ-n_endog,m_states),
                                FF - dot(dot(JJ,CC_plus),AA)))
#print('Psi_mat = ', Psi_mat)
#print('Psi_mat shape =', Psi_mat.shape)
#print('Psi_mat dim =', np.ndim(Psi_mat))
#print('Psi_mat rank =', np.linalg.matrix_rank(Psi_mat))
#print(' ')

Theta_mat = np.vstack((dot(CC_0,BB),
                                       dot(dot(KK,CC_plus),BB) - HH))
#print('Theta_mat = ', Theta_mat)
#print('Theta_mat shape =', Theta_mat.shape)
#print('Theta_mat dim =', np.ndim(Theta_mat))
#print('Theta_mat rank =', np.linalg.matrix_rank(Theta_mat))  
#print(' ')

Xi_mat = np.vstack((np.hstack((Gamma_mat, Theta_mat)),
                                 np.hstack((eye(m_states),zeros(m_states**2).reshape(m_states,m_states)))))
#print('Xi_mat = ', Xi_mat)
#print('Xi_mat shape =', Xi_mat.shape)
#print('Xi_mat dim =', np.ndim(Xi_mat))
#print('Xi_mat rank =', np.linalg.matrix_rank(Xi_mat))  
#print(np.ndim(eye(m_states)))
#print((eye(m_states)).shape)
#print((zeros(m_states**2).reshape(m_states,m_states)).shape)
#print(' ')

Delta_mat = np.vstack((np.hstack((Psi_mat, zeros(m_states**2).reshape(m_states,m_states))),
                    np.hstack((zeros(m_states**2).reshape(m_states,m_states),eye(m_states)))))
#print('Delta_mat = ', Delta_mat)
#print('Delta_mat shape =', Delta_mat.shape)
#print('Delta_mat dim =', np.ndim(Delta_mat))
#print('Delta_mat rank =', np.linalg.matrix_rank(Delta_mat))
#print(' ')



##  uhlig_solver_abreviated.ipynb and uhlig_solver_QZ.ipynb 
##  deviate from here until we arrive at a solution for PP
##  This subsection is due to Kerk Phillips and Yulong Li


def qzswitch(i, A, B, Q, Z):  # (c) Chris Sims
    
    a = A[i-1,i-1]
    d = B[i-1,i-1]
    b = A[i-1,i]
    e = B[i-1,i]
    c = A[i,i]
    f = B[i,i]
  
    wz = hstack((dot(c,e)-dot(f,b), (dot(c,d)-dot(f,a)).conj().T))
    xy = hstack(((dot(b,d)-dot(e,a)).conj().T, (dot(c,d)-dot(f,a)).conj().T))

    n = np.sqrt(dot(wz,wz.conj().T))
    m = np.sqrt(dot(xy,xy.conj().T))
    
    if n == 0:
        print ('qzswitch(): Inputs are unchanged.')
        return A, B, Q, Z
   
    else:
        wz = wz/n
        xy = xy/m
        wz = vstack(( wz, hstack((-wz[1].conj().T, wz[0].conj().T)) ))
        xy = vstack(( xy, hstack((-xy[1].conj().T, xy[0].conj().T)) ))
        A[i-1:i+1,:] = xy.dot(A[i-1:i+1,:])
        B[i-1:i+1,:] = xy.dot(B[i-1:i+1,:])
        A[:,i-1:i+1] = A[:,i-1:i+1].dot(wz)
        B[:,i-1:i+1] = B[:,i-1:i+1].dot(wz)
        Z[:,i-1:i+1] = Z[:,i-1:i+1].dot(wz)
        Q[i-1:i+1,:] = xy.dot(Q[i-1:i+1,:])
    
    return A, B, Q, Z


def qzdiv(stake, A, B, Q, Z):  # (c) Chris Sims
    
    n, jnk = A.shape

    root = abs(vstack((np.diag(A), np.diag(B))).T)
    tmp = (root[:,0]<1.e-13).astype(int)
    
    root[:,0] = root[:,0]- tmp *(root[:,0]+root[:,1])
    root[:,1] = root[:,1]/root[:,0]
    
    for i in range(n,0,-1):
        m=0
        
        for j in range(i,0,-1):
            if (root[j-1,1] > stake or root[j-1,1] < -.1):
                m=j
                break
                
        if m==0:
            print ('qzdiv(): Inputs are unchanged')
            return A, B, Q, Z
        
        for k in range(m,i,1):
            
            A, B, Q, Z = qzswitch(k,A,B,Q,Z)
            
            tmp = root[k-1,1]
            root[k-1,1] = root[k,1]
            root[k,1] = tmp
            
    return A, B, Q, Z


Delta_up,Xi_up,UUU,VVV = la.qz(Delta_mat,Xi_mat, output='complex')

UUU = UUU.T

Xi_eigval = np.diag( np.diag(Xi_up)/np.maximum(np.diag(Delta_up),TOL))
Xi_sortabs = np.sort(abs(np.diag(Xi_eigval)))
Xi_sortindex = np.argsort(abs(np.diag(Xi_eigval)))
Xi_sortval = Xi_eigval[Xi_sortindex, Xi_sortindex]
Xi_select = np.arange(0, nx)

stake = max(abs(Xi_sortval[Xi_select])) + TOL


Delta_up, Xi_up, UUU, VVV = qzdiv(stake,Delta_up,Xi_up,UUU,VVV)
   
VVV = VVV.conj().T
VVV_2_1 = VVV[nx : 2*nx, 0 : nx]
VVV_2_2 = VVV[nx : 2*nx, nx :2*nx]
UUU_2_1 = UUU[nx : 2*nx, 0 : nx]
VVV = VVV.conj().T


PP = - la.solve(VVV_2_1 ,VVV_2_2) 
PP_imag = np.imag(PP)
PP = np.real(PP)
#print('PP = ', PP)
#print('PP =', PP.shape)
#print('PP dim =', np.ndim(PP))
#print('PP rank =', np.linalg.matrix_rank(PP))
#print(' ')

##  This ends the section based on qzdiv.m and 
##  qzswitch.m  by Sims

##  uhlig_solver_abreviated.ipynb and uhlig_solver_QZ.ipynb 
##  are again identical from here 


RR = - dot(CC_plus, (dot(AA, PP) + BB))
#print('RR = ', RR)
#print('RR =', RR.shape)
#print('RR dim =', np.ndim(RR))
#print('RR rank =', np.linalg.matrix_rank(RR))
#print(' ')

VV = np.vstack((hstack((kron(eye(k_exog), AA), \
                           kron(eye(k_exog), CC))), hstack((kron(NN.T, FF) +\
                           kron(eye(k_exog), dot(FF, PP) + dot(JJ, RR) + GG),\
                           kron(NN.T, JJ) + kron(eye(k_exog), KK)))))
#rank_count = k_exog*(m_states+n_endog)
#print('VV = ', VV)
#print('VV =', VV.shape)
#print('VV dim =', np.ndim(VV))
#print('VV rank =', np.linalg.matrix_rank(VV))
#print('rank should be = ',rank_count)
#print(' ')

LLNN_plus_MM = dot(LL,NN) + MM
#print('LLNN_plus_MM = ', LLNN_plus_MM)
#print('LLNN_plus_MM shape =', LLNN_plus_MM.shape)
#print('LLNN_plus_MM dim =', np.ndim(LLNN_plus_MM))
#print('LLNN_plus_MM rank =', np.linalg.matrix_rank(LLNN_plus_MM))
#print('   ')

counter_i = 0
DD_flat = DD[:,0]
for counter_i in range(1,DD.shape[1]):
    DD_col = DD[:,counter_i] 
    DD_flat = np.vstack((DD_flat,DD_col))
#print('DD_flat = ', DD_flat)
#print('DD_flat shape=', DD_flat.shape) 
#print('DD_flat dim =', np.ndim(DD_flat)) 
#print('DD_flat rank =', np.linalg.matrix_rank(DD_flat))
#print('  ')

counter_i = 0
LLNN_plus_MM_flat = LLNN_plus_MM[:,0]
for counter_i in range(1,LLNN_plus_MM.shape[1]):
    LLNN_plus_MM_col = LLNN_plus_MM[:,counter_i] 
    LLNN_plus_MM_flat = np.vstack((LLNN_plus_MM_flat,LLNN_plus_MM_col))
#print('LLNN_plus_MM_flat', LLNN_plus_MM_flat)
#print('LLNN_plus_MM_flat shape =', LLNN_plus_MM_flat.shape) 
#print('LLNN_plus_MM_flat dim =', np.ndim(LLNN_plus_MM_flat)) 
#print('LLNN_plus_MM_flat =', np.linalg.matrix_rank(LLNN_plus_MM_flat))
#print('  ')

denominator = np.vstack((DD_flat,LLNN_plus_MM_flat))
#print('denominator = ',denominator)
#print('denominator shape =', denominator.shape) 
#print('denominator dim =', np.ndim(denominator)) 
#print('denominator=', np.linalg.matrix_rank(denominator))
#print('  ')

QQSS_vec = - la.solve(VV,denominator) 
#print('QQSS_vec = ', QQSS_vec) 
#print('QQSS_vec =', QQSS_vec.shape) 
#print('QQSS_vec dim =', np.ndim(QQSS_vec)) 
#print('QQSS_vec rank =', np.linalg.matrix_rank(QQSS_vec))
#print('QQSS_vec max element = ', np.max(np.absolute(QQSS_vec)))
#print('This should not be infinite in value.')
#print('  ')

QQ = np.reshape(np.matrix(QQSS_vec[0:m_states * k_exog , 0]), (m_states, k_exog), 'F')
#print('QQ = ', QQ)
#print('QQ =', QQ.shape) 
#print('QQ dim =', np.ndim(QQ)) 
#print('QQ rank =', np.linalg.matrix_rank(QQ))
#print('  ')

SS = np.reshape(QQSS_vec[(m_states * k_exog ):((m_states + n_endog) * k_exog), 0], (n_endog, k_exog), 'F')
#print('SS = ', SS)
#print('SS =', SS.shape) 
#print('SS dim =', np.ndim(SS)) 
#print('SS rank =', np.linalg.matrix_rank(SS))
#print('  ')

WW = np.vstack((np.hstack((eye(m_states), zeros((m_states, k_exog)))),
                             np.hstack((dot(RR, np.linalg.pinv(PP)), (SS - dot(dot(RR, np.linalg.pinv(PP)), QQ)))),
                             np.hstack((zeros((k_exog, m_states)), eye(k_exog)))))
#print('WW = ', WW)
#print('WW =', WW.shape) 
#print('WW dim =', np.ndim(WW)) 
#print('WW rank =', np.linalg.matrix_rank(WW))
#print('  ')

print('Done. You have the matrices PP QQ  RR  SS  WW.')

